# AI Mathematical Olympiad (AIMO) Progress Prize 3

## Competition Overview

This is the **third AIMO Progress Prize competition**, building upon:
- **First Prize (July 2024)**: Won by Project Numina
- **Second Prize (April 2025)**: Won by Nvidia's NemoSkills team

### Key Features of AIMO3:
- **Significantly increased problem difficulty**: 110 problems spanning algebra, combinatorics, geometry, and number theory
- **Problem difficulty range**: From National Olympiad level to IMO standard (pinnacle of high school mathematics)
- **New submission format**: Dual-answer system with penalized accuracy scoring
- **Expanded prize pool**: Part of $10mn AIMO Prize fund
- **Industry-leading compute resources** for participants
- **Auxiliary prizes** to reward community contributions
- **Updated rules** for using open-source LLMs

### The Challenge

The ability to reason mathematically is a critical milestone for AI. Current AI capabilities show a significant gap between closed-source and open-source models:
- Closed-source models achieved **gold medal performance** at 2025 IMO
- OpenAI x AIMO eval (March 2025): Commercial models solved **50/50 AIMO2 problems**
- Highest Kaggle score was only **34/50**, demonstrating the gap

AIMO3 is designed to **accelerate open-source progress** and close this gap.

### Problem Design

- **100% original problems**: Created by international team of problem solvers
- **Zero risk of train-test contamination**
- **"AI hard" design**: Tested against current open LLMs' capabilities
- **Genuine mathematical reasoning required**: Answer-only testing is robust

## Evaluation Criteria

### Public Leaderboard (During Competition)
- Evaluated on **public test set only**
- Score = **unnormalized accuracy** (number of correct answers)

### Private Leaderboard (Final Evaluation)
- Submission notebooks run **twice** over private test set
- Predictions concatenated into single submission file
- **Penalized accuracy scoring**:
  - **Both answers correct**: 1.0 point
  - **One answer correct**: 0.5 points
  - **Neither answer correct**: 0.0 points
- **Overall score**: Sum of scores for all problems

### Answer Format
- All ground-truth labels are **integers between 0 and 99999** (inclusive)
- Any modulo calculation is **explicitly stated** in problem statement
- **No implicit modulo 1000** (change from AIMO1 and AIMO2)

---

## Approach

This notebook implements a solution using:
1. **Data exploration and preprocessing**
2. **LLM-based mathematical reasoning** with prompt engineering
3. **Multiple solving strategies** to generate diverse solutions
4. **Dual-answer submission** system for robust evaluation

## 1. Setup and Imports

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import json
import re
import warnings
from pathlib import Path
from typing import List, Tuple, Dict, Optional
import os

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully!")

## 2. Data Loading and Exploration

Load the competition data files. The data directory should contain:
- `train.csv`: Training problems with solutions
- `test.csv`: Test problems to solve
- `sample_submission.csv`: Submission format template

In [ ]:
# Define data path - Update this to your local data directory
DATA_PATH = Path("/kaggle/input/ai-mathematical-olympiad-progress-prize-3")

# Alternative path for local development
# DATA_PATH = Path("C:/Users/gopeami/OneDrive - Vesuvius/Desktop/PhD13- 2025-2026/ML Practice/Kaggle Compettition/AI Mathematical Olympiad/ai-mathematical-olympiad-progress-prize-3")

# Check if data directory exists
if DATA_PATH.exists():
    print(f"Data directory found: {DATA_PATH}")
    print(f"\nFiles in directory:")
    for file in DATA_PATH.iterdir():
        print(f"  - {file.name}")
else:
    print(f"⚠️ Data directory not found: {DATA_PATH}")
    print("Please update DATA_PATH to your local data directory")

In [ ]:
# Load the datasets
try:
    train_df = pd.read_csv(DATA_PATH / "train.csv")
    test_df = pd.read_csv(DATA_PATH / "test.csv")
    sample_submission = pd.read_csv(DATA_PATH / "sample_submission.csv")
    
    print("✓ Data loaded successfully!\n")
    print(f"Training set shape: {train_df.shape}")
    print(f"Test set shape: {test_df.shape}")
    print(f"Sample submission shape: {sample_submission.shape}")
    
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    print("Creating dummy data for demonstration...")
    
    # Create dummy dataframes for development
    train_df = pd.DataFrame({
        'id': range(5),
        'problem': ['Sample problem ' + str(i) for i in range(5)],
        'answer': [12345, 67890, 11111, 22222, 33333]
    })
    test_df = pd.DataFrame({
        'id': range(10),
        'problem': ['Test problem ' + str(i) for i in range(10)]
    })
    sample_submission = pd.DataFrame({
        'id': range(10),
        'answer': [0] * 10
    })

In [ ]:
# Explore the training data
print("=" * 80)
print("TRAINING DATA OVERVIEW")
print("=" * 80)
print(f"\nColumns: {list(train_df.columns)}")
print(f"\nFirst few rows:")
print(train_df.head())

if 'answer' in train_df.columns:
    print(f"\n\nAnswer Statistics:")
    print(train_df['answer'].describe())
    print(f"\nAnswer range: {train_df['answer'].min()} to {train_df['answer'].max()}")
    print(f"All answers in valid range (0-99999): {train_df['answer'].between(0, 99999).all()}")

In [ ]:
# Explore the test data
print("=" * 80)
print("TEST DATA OVERVIEW")
print("=" * 80)
print(f"\nColumns: {list(test_df.columns)}")
print(f"\nFirst few rows:")
print(test_df.head())

print(f"\n\nTotal problems to solve: {len(test_df)}")

In [ ]:
# Check sample submission format
print("=" * 80)
print("SAMPLE SUBMISSION FORMAT")
print("=" * 80)
print(f"\nColumns: {list(sample_submission.columns)}")
print(f"\nFirst few rows:")
print(sample_submission.head(10))

print(f"\n\nSubmission requirements:")
print(f"  - Each problem ID should appear exactly once")
print(f"  - Answers must be integers between 0 and 99999")
print(f"  - Notebook will be run TWICE and predictions concatenated for dual-answer scoring")

## 3. Mathematical Reasoning Framework

This section implements the core problem-solving approach using:
- **Prompt Engineering**: Structured prompts for mathematical problem solving
- **Answer Extraction**: Robust parsing of numerical answers
- **Multiple Strategies**: Different solving approaches for diversity

### Key Strategies:
1. **Chain-of-Thought Reasoning**: Step-by-step mathematical deduction
2. **Multiple Approaches**: Algebra, combinatorics, geometry, number theory
3. **Verification**: Answer validation and range checking

In [ ]:
class MathProblemSolver:
    """
    Mathematical problem solver using structured reasoning approaches.
    Designed for AIMO3 competition requirements.
    """
    
    def __init__(self):
        self.valid_range = (0, 99999)
        
    def extract_answer(self, text: str) -> Optional[int]:
        """
        Extract numerical answer from solution text.
        Returns integer between 0 and 99999, or None if not found.
        """
        if not text:
            return None
        
        # Look for common answer patterns
        patterns = [
            r'(?:answer|solution|result)(?:\s+is)?(?:\s*:)?\s*(\d+)',
            r'(?:equals?|=)\s*(\d+)',
            r'(?:therefore|thus|hence)(?:,)?\s*(?:the answer is)?\s*(\d+)',
            r'\b(\d+)\s*(?:is the answer|is the solution)',
            r'final answer:\s*(\d+)',
            r'\\boxed\{(\d+)\}',  # LaTeX boxed answer
            r'\$(\d+)\$',  # LaTeX math mode
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                try:
                    answer = int(match.group(1))
                    if self.valid_range[0] <= answer <= self.valid_range[1]:
                        return answer
                except (ValueError, IndexError):
                    continue
        
        # Fallback: look for last number in text
        numbers = re.findall(r'\b(\d+)\b', text)
        if numbers:
            for num in reversed(numbers):
                try:
                    answer = int(num)
                    if self.valid_range[0] <= answer <= self.valid_range[1]:
                        return answer
                except ValueError:
                    continue
        
        return None
    
    def validate_answer(self, answer: Optional[int]) -> int:
        """
        Validate and sanitize answer.
        Returns valid integer or 0 as fallback.
        """
        if answer is None:
            return 0
        
        try:
            answer = int(answer)
            # Clamp to valid range
            return max(self.valid_range[0], min(answer, self.valid_range[1]))
        except (ValueError, TypeError):
            return 0
    
    def create_prompt(self, problem: str, strategy: str = "standard") -> str:
        """
        Create structured prompt for mathematical problem solving.
        
        Args:
            problem: The mathematical problem statement
            strategy: Solving strategy ('standard', 'detailed', 'creative')
        """
        if strategy == "detailed":
            prompt = f"""You are a world-class mathematician solving an International Mathematical Olympiad problem.

Problem: {problem}

Please solve this problem step-by-step:
1. Understand the problem and identify what is being asked
2. Identify the mathematical concepts and techniques needed
3. Work through the solution systematically with clear reasoning
4. Verify your answer makes sense
5. State the final numerical answer clearly

Remember: The answer must be an integer between 0 and 99999.
If modulo arithmetic is needed, it will be explicitly stated in the problem.

Solution:"""
        
        elif strategy == "creative":
            prompt = f"""You are solving a challenging mathematical olympiad problem that requires creative thinking.

Problem: {problem}

Approach this problem by:
1. Consider multiple solution strategies
2. Look for patterns or special properties
3. Use elegant mathematical insights
4. Double-check your calculations
5. Provide the final integer answer (0-99999)

Solution:"""
        
        else:  # standard
            prompt = f"""Solve the following mathematical problem and provide the numerical answer.

Problem: {problem}

Provide a clear solution with the final answer as an integer between 0 and 99999.

Solution:"""
        
        return prompt
    
    def solve_problem_symbolic(self, problem: str) -> Optional[int]:
        """
        Attempt to solve problem using symbolic computation (sympy).
        This is a placeholder for integration with symbolic math libraries.
        """
        # This would integrate with SymPy or similar libraries
        # For now, return None to indicate symbolic solving not available
        return None
    
    def solve_problem_numerical(self, problem: str) -> Optional[int]:
        """
        Attempt to solve problem using numerical methods.
        This is a placeholder for numerical computation approaches.
        """
        # This would use NumPy, SciPy, or similar for numerical solutions
        # For now, return None
        return None

# Initialize solver
solver = MathProblemSolver()
print("✓ Mathematical Problem Solver initialized successfully!")

In [ ]:
# Test the solver with a sample problem
sample_problem = "What is the sum of the first 100 positive integers?"

print("Testing MathProblemSolver with sample problem:")
print(f"\nProblem: {sample_problem}")
print("\n" + "=" * 80)

# Test prompt generation
prompt = solver.create_prompt(sample_problem, strategy="detailed")
print("\nGenerated Prompt (first 300 chars):")
print(prompt[:300] + "...")

# Test answer extraction
test_solutions = [
    "The answer is 5050",
    "Therefore, the sum equals 5050.",
    "Using the formula n(n+1)/2, we get 100(101)/2 = 5050",
    "Final answer: \\boxed{5050}",
    "The result is $5050$"
]

print("\n" + "=" * 80)
print("Testing Answer Extraction:")
for i, solution in enumerate(test_solutions, 1):
    extracted = solver.extract_answer(solution)
    print(f"\n{i}. Solution: '{solution}'")
    print(f"   Extracted answer: {extracted}")

## 4. Solution Generation Pipeline

This section implements the complete solution pipeline:

1. **Problem Processing**: Load and prepare test problems
2. **Multi-Strategy Solving**: Apply different solving approaches
3. **Answer Aggregation**: Combine results from multiple strategies
4. **Dual-Answer System**: Prepare for the two-run submission format

### Dual-Answer Strategy:
Since notebooks are run **twice** and predictions concatenated:
- **Run 1**: Use primary solving strategy
- **Run 2**: Use alternative strategy or randomized approach
- This maximizes chances of getting at least one correct answer (0.5 points)

In [ ]:
def solve_with_llm_placeholder(problem: str, strategy: str = "standard") -> int:
    """
    Placeholder for LLM-based solving.
    
    In actual implementation, this would:
    1. Call an LLM API (OpenAI, Anthropic, Gemini, etc.)
    2. Use the generated prompt
    3. Extract the answer from the response
    
    For now, returns a random valid answer for demonstration.
    """
    # This is where you would integrate with your LLM of choice
    # Example with OpenAI API:
    # response = openai.ChatCompletion.create(
    #     model="gpt-4",
    #     messages=[{"role": "user", "content": solver.create_prompt(problem, strategy)}]
    # )
    # answer_text = response.choices[0].message.content
    # return solver.extract_answer(answer_text)
    
    # Placeholder: return a deterministic hash-based answer for demo
    import hashlib
    hash_val = int(hashlib.md5(problem.encode()).hexdigest(), 16)
    return hash_val % 100000  # Keep in valid range

def solve_problem_multi_strategy(problem_id: int, problem: str) -> Dict[str, int]:
    """
    Solve a problem using multiple strategies and return all answers.
    
    Returns:
        Dictionary with strategy names as keys and answers as values
    """
    results = {}
    
    # Strategy 1: Standard approach
    results['standard'] = solve_with_llm_placeholder(problem, strategy="standard")
    
    # Strategy 2: Detailed step-by-step
    results['detailed'] = solve_with_llm_placeholder(problem, strategy="detailed")
    
    # Strategy 3: Creative approach
    results['creative'] = solve_with_llm_placeholder(problem, strategy="creative")
    
    # Strategy 4: Symbolic computation (if available)
    symbolic_answer = solver.solve_problem_symbolic(problem)
    if symbolic_answer is not None:
        results['symbolic'] = symbolic_answer
    
    # Strategy 5: Numerical methods (if available)
    numerical_answer = solver.solve_problem_numerical(problem)
    if numerical_answer is not None:
        results['numerical'] = numerical_answer
    
    return results

def select_best_answer(results: Dict[str, int]) -> int:
    """
    Select the best answer from multiple strategies.
    
    Strategy: Use majority voting, or select most frequent answer.
    Fallback to first available answer.
    """
    if not results:
        return 0
    
    # Count frequency of each answer
    from collections import Counter
    answer_counts = Counter(results.values())
    
    # Return most common answer
    most_common = answer_counts.most_common(1)[0][0]
    return solver.validate_answer(most_common)

print("✓ Solution generation pipeline functions defined!")

In [ ]:
# Test the multi-strategy solving on a sample problem
if len(test_df) > 0:
    sample_idx = 0
    sample_id = test_df.iloc[sample_idx]['id']
    sample_problem = test_df.iloc[sample_idx]['problem']
    
    print(f"Testing multi-strategy solving:")
    print(f"\nProblem ID: {sample_id}")
    print(f"Problem: {sample_problem[:200]}..." if len(sample_problem) > 200 else f"Problem: {sample_problem}")
    print("\n" + "=" * 80)
    
    # Solve using multiple strategies
    strategy_results = solve_problem_multi_strategy(sample_id, sample_problem)
    
    print("\nResults from different strategies:")
    for strategy, answer in strategy_results.items():
        print(f"  {strategy:15s}: {answer}")
    
    # Select best answer
    best_answer = select_best_answer(strategy_results)
    print(f"\n{'Selected answer':15s}: {best_answer}")
    print("=" * 80)

## 5. Generate Predictions for All Test Problems

Now we'll process all test problems and generate predictions. 

**Important Notes:**
- In a production setting, replace `solve_with_llm_placeholder` with actual LLM API calls
- Consider using local open-source models (Llama, Mistral, etc.) for Kaggle submission
- Implement caching to avoid redundant API calls
- Add error handling and retry logic for API failures

In [ ]:
def generate_all_predictions(test_data: pd.DataFrame, 
                           use_multi_strategy: bool = True,
                           verbose: bool = True) -> pd.DataFrame:
    """
    Generate predictions for all test problems.
    
    Args:
        test_data: DataFrame with 'id' and 'problem' columns
        use_multi_strategy: If True, use multiple strategies and vote
        verbose: If True, print progress
    
    Returns:
        DataFrame with 'id' and 'answer' columns
    """
    predictions = []
    
    total_problems = len(test_data)
    
    for idx, row in test_data.iterrows():
        problem_id = row['id']
        problem = row['problem']
        
        if verbose and (idx % 10 == 0 or idx == total_problems - 1):
            print(f"Processing problem {idx + 1}/{total_problems} (ID: {problem_id})...")
        
        if use_multi_strategy:
            # Use multiple strategies and select best answer
            results = solve_problem_multi_strategy(problem_id, problem)
            answer = select_best_answer(results)
        else:
            # Use single strategy
            answer = solve_with_llm_placeholder(problem, strategy="standard")
            answer = solver.validate_answer(answer)
        
        predictions.append({
            'id': problem_id,
            'answer': answer
        })
    
    return pd.DataFrame(predictions)

# Generate predictions for all test problems
print("=" * 80)
print("GENERATING PREDICTIONS FOR ALL TEST PROBLEMS")
print("=" * 80)
print(f"\nTotal problems to solve: {len(test_df)}")
print("\nStarting prediction generation...")
print("-" * 80)

predictions_df = generate_all_predictions(test_df, use_multi_strategy=True, verbose=True)

print("-" * 80)
print("✓ Prediction generation complete!")
print(f"\nGenerated predictions for {len(predictions_df)} problems")
print(f"\nFirst few predictions:")
print(predictions_df.head(10))

## 6. Validate and Prepare Submission

Final validation steps before creating submission file:
1. Check all problem IDs are present
2. Verify all answers are in valid range (0-99999)
3. Ensure correct format matches sample_submission.csv

In [ ]:
# Validation checks
print("=" * 80)
print("SUBMISSION VALIDATION")
print("=" * 80)

# Check 1: All test IDs present
test_ids = set(test_df['id'])
pred_ids = set(predictions_df['id'])

missing_ids = test_ids - pred_ids
extra_ids = pred_ids - test_ids

print(f"\n✓ Check 1: Problem ID coverage")
print(f"  Test problems: {len(test_ids)}")
print(f"  Predictions: {len(pred_ids)}")
print(f"  Missing IDs: {len(missing_ids)}")
print(f"  Extra IDs: {len(extra_ids)}")

if missing_ids:
    print(f"  ⚠️ Warning: Missing predictions for IDs: {missing_ids}")
if extra_ids:
    print(f"  ⚠️ Warning: Extra predictions for IDs: {extra_ids}")

# Check 2: Answer range validation
print(f"\n✓ Check 2: Answer range (0-99999)")
print(f"  Min answer: {predictions_df['answer'].min()}")
print(f"  Max answer: {predictions_df['answer'].max()}")

invalid_answers = predictions_df[
    (predictions_df['answer'] < 0) | (predictions_df['answer'] > 99999)
]
print(f"  Invalid answers: {len(invalid_answers)}")

if len(invalid_answers) > 0:
    print(f"  ⚠️ Warning: Found invalid answers:")
    print(invalid_answers)

# Check 3: Data types
print(f"\n✓ Check 3: Data types")
print(f"  ID type: {predictions_df['id'].dtype}")
print(f"  Answer type: {predictions_df['answer'].dtype}")

# Check 4: No missing values
print(f"\n✓ Check 4: Missing values")
print(f"  Missing IDs: {predictions_df['id'].isna().sum()}")
print(f"  Missing answers: {predictions_df['answer'].isna().sum()}")

# Check 5: Format matches sample submission
print(f"\n✓ Check 5: Format verification")
print(f"  Sample submission columns: {list(sample_submission.columns)}")
print(f"  Predictions columns: {list(predictions_df.columns)}")
print(f"  Columns match: {list(sample_submission.columns) == list(predictions_df.columns)}")

print("\n" + "=" * 80)
print("Validation complete!")
print("=" * 80)

## 7. Create Submission File

Create the final submission file in the correct format.

**Important Reminder:**
- The submission notebook will be run **TWICE** by Kaggle
- Predictions from both runs will be concatenated
- This enables the dual-answer scoring system:
  - Both correct = 1.0 point
  - One correct = 0.5 points  
  - Neither correct = 0.0 points

In [ ]:
# Prepare submission DataFrame
submission = predictions_df.copy()

# Ensure correct column order
submission = submission[['id', 'answer']]

# Sort by ID for consistency
submission = submission.sort_values('id').reset_index(drop=True)

# Display final submission
print("=" * 80)
print("FINAL SUBMISSION")
print("=" * 80)
print(f"\nSubmission shape: {submission.shape}")
print(f"\nFirst 10 rows:")
print(submission.head(10))
print(f"\nLast 10 rows:")
print(submission.tail(10))
print("\n" + "=" * 80)

# Save to CSV
submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)

print(f"\n✓ Submission saved to: {submission_path}")
print(f"  File size: {os.path.getsize(submission_path)} bytes")
print("\n" + "=" * 80)

## 8. Summary and Next Steps

### What This Notebook Does:

1. ✅ **Loads competition data** (train.csv, test.csv, sample_submission.csv)
2. ✅ **Implements mathematical problem solver** with:
   - Structured prompt engineering
   - Answer extraction from text
   - Validation and sanitization
3. ✅ **Multi-strategy solving approach**:
   - Standard reasoning
   - Detailed step-by-step
   - Creative problem-solving
   - Symbolic computation (placeholder)
   - Numerical methods (placeholder)
4. ✅ **Generates predictions** for all test problems
5. ✅ **Validates submission** format and answer ranges
6. ✅ **Creates submission.csv** file

### To Improve Performance:

#### 1. **Integrate Real LLM**
Replace `solve_with_llm_placeholder` with actual LLM calls:
```python
# Example with OpenAI
import openai
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": solver.create_prompt(problem)}]
)
```

#### 2. **Use Open-Source Models**
For Kaggle submission, consider local models:
- **Llama 3** (70B or 405B)
- **Qwen 2.5** (Math specialized)
- **DeepSeek-Math**
- **Mistral Large**

#### 3. **Implement Symbolic Math**
Integrate SymPy for exact symbolic computation:
```python
import sympy as sp
# Parse problem and solve symbolically
```

#### 4. **Add Verification**
- Check answer validity within problem context
- Verify calculations
- Test edge cases

#### 5. **Optimize Prompts**
- Include few-shot examples
- Add chain-of-thought prompting
- Use problem-type specific prompts

#### 6. **Ensemble Methods**
- Combine multiple models
- Use majority voting
- Implement confidence scoring

### Competition Strategy:

**Dual-Run Optimization:**
- Run 1: Use most reliable strategy
- Run 2: Use diverse/alternative strategy
- This maximizes partial credit (0.5 points)

**Focus Areas:**
1. **Algebra**: Pattern recognition, equation solving
2. **Combinatorics**: Counting, probability
3. **Geometry**: Spatial reasoning, coordinate geometry
4. **Number Theory**: Modular arithmetic, divisibility

### Resources:

- **Competition Page**: ai-mathematical-olympiad-progress-prize-3
- **Reference Problems**: Check the PDF in data folder
- **Previous Competitions**: Study AIMO1 and AIMO2 winning solutions

---

**Good luck with the competition! 🎯**